In [2]:
from torchtext import data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from dataset import DataHandler, BertField
import torch.nn as nn
import torch
from model import BiAttention, EncoderRNN, SelfAttention
import os
import torchtext
from tensorboardX import SummaryWriter

os.environ["CUDA_VISIBLE_DEVICES"] = '2'

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
train_examples_path = './train_examples.pt'
val_examples_path = './val_examples.pt'

In [4]:
tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased-vocab.txt', do_lower_case=True)

bert_field = BertField(tokenizer)
multi_bert_field = data.NestedField(bert_field)



word_field = data.Field(batch_first=True, sequential=True, tokenize=tokenizer.tokenize, lower=True) # query
multi_word_field = data.NestedField(word_field) 

word_field_sup = data.Field(batch_first=True, sequential=True, tokenize=tokenizer.tokenize, lower=True, fix_length=320)
multi_word_field_sup = data.NestedField(word_field_sup) 

bert_field_sup = BertField(tokenizer, fix_length=320)
multi_bert_field_sup = data.NestedField(bert_field_sup)

raw = data.RawField()
raw.is_target = False

label_field = data.Field(sequential=False, is_target=True, use_vocab=False)

dict_field = {
    'id': ('id', raw),
    'supports': [('s_glove', multi_word_field_sup), ('s_bert', multi_bert_field_sup)],
    'query': [('q_glove', word_field), ('q_bert', bert_field)],
    'answer': [('a_glove', word_field), ('a_bert', bert_field)],
    'candidates': [('c_glove', multi_word_field), ('c_bert', multi_bert_field)],
    'label': ('label', label_field)

}

In [5]:
data_handler = DataHandler(train_examples_path, val_examples_path, dict_field)

load examples.pt  :./train_examples.pt, ./val_examples.pt


In [6]:
multi_word_field_sup.build_vocab(data_handler.trainset, data_handler.valset, vectors=torchtext.vocab.GloVe(dim=300,name='6B') )
word_field.vocab = multi_word_field_sup.vocab
word_field.include_lengths = True

In [7]:
train_iter = data_handler.get_train_iter(batch_size=1)
val_iter = data_handler.get_val_iter(batch_size=1)

### Embedding

这一层需要频繁的改动，所以暂时不放在py文件中

In [8]:
class EmbeddingLayer(nn.Module):
    
    def __init__(self, word_field, bert_model_path='./bert-base-uncased/', use_all=False):
        super(EmbeddingLayer, self).__init__()
        self.word_embedding_layer = nn.Embedding.from_pretrained(embeddings=word_field.vocab.vectors)
        
        model = BertModel.from_pretrained(bert_model_path)   
        self.bert_model = model
        
        self.use_all = use_all
        self.freeze()
        
    def freeze(self):
        for param in self.bert_model.parameters():
            param.requires_grad = False
        self.word_embedding_layer.weight.requires_grad = False
        
    def forward(self, word_tokens, bert_tokens, input_mask=None):
        '''
        input:
            x: [batch_size, seg_len]
        
        return embeddings: [batch_size, seq_len, glove_dim + bert_dim]    
        '''
        word_embeddings = self.word_embedding_layer(word_tokens)
        
        # encoded_layers: [batch_size, seq_len, bert_embedding_dim] * num_of_layers
        encoded_layers, _ = self.bert_model(bert_tokens, attention_mask=input_mask)
        
        bert_embeddings = torch.zeros_like(encoded_layers[-1])
        if self.use_all:
            for layer in encoded_layers:
                bert_embeddings += layer
            bert_embeddings /= len(encoded_layers)
        else:
            bert_embeddings += encoded_layers[-1]
        
        out = torch.cat([word_embeddings, bert_embeddings], dim=-1)
        return out
        

In [9]:
class SimpleQANet(nn.Module):
    
    def __init__(self, config, word_field):
        super(SimpleQANet, self).__init__()
        self.config = config
        self.use_cuda = config.use_cuda
        
        self.embedding_layer = EmbeddingLayer(word_field, config.bert_path, config.use_all)
        self.rnn = EncoderRNN(config.word_dim + config.bert_dim, config.hidden, 1, True, True, 0.2, False)
        
        self.qc_att = BiAttention(config.hidden*2, 0.2)
        self.linear_1 = nn.Sequential(
                nn.Linear(config.hidden*8, config.hidden),
                nn.ReLU()
        )    
        
        self.rnn_2 = EncoderRNN(config.hidden, config.hidden, 1, False, True, 0.2, False)
        
        self.self_att = SelfAttention(config.hidden*2, config.hidden*2, 0.2)       
        self.self_att_2 = SelfAttention(config.hidden*2, config.hidden*2, 0.2)        
        
        self.self_att_c = SelfAttention(config.hidden*2, config.hidden*2, 0.2)        
        
        
    def forward(self, batch):
        q_glove, _ = batch.q_glove
        q_bert = batch.q_bert
        s_glove = batch.s_glove
        s_bert = batch.s_bert
        c_glove = batch.c_glove
        c_bert = batch.c_bert
        
        if self.use_cuda:
            q_glove = q_glove.cuda()
            q_bert = q_bert.cuda()
            s_glove = s_glove.cuda().squeeze(0)
            s_bert = s_bert.cuda().squeeze(0)
            c_glove = c_glove.cuda().squeeze(0)
            c_bert = c_bert.cuda().squeeze(0)
            
        # Embedding 
        context_mask = (s_bert > 0).float()
        ques_mask = (q_bert > 0).float()
        
        q_out = self.embedding_layer(q_glove, q_bert)
        s_out = self.embedding_layer(s_glove, s_bert, context_mask)
        c_out = self.embedding_layer(c_glove, c_bert)

        q_out = self.rnn(q_out)
        c_out = self.rnn(c_out)
        
        s_out = self.rnn(s_out)

        # bi-attention on supports and  question
        context_mask = (c_bert.squeeze() > 0).float()
        ques_mask = (q_bert > 0).float()
        
        support_len = s_out.size(0)
        q_out = q_out.expand(support_len, q_out.size(1), q_out.size(2))
        ques_mask = ques_mask.expand(support_len, q_out.size(1))        
        
        # s_out:[supports_len, seq_len, hidden*2], q_out: [support_len, seq_len, hidden*2]
        output = self.qc_att(s_out, q_out, ques_mask)
        output = self.linear_1(output)
        output = self.rnn_2(output)
        
        # self-attention pooling 
        # [support_len, hidden*2]
        output = self.self_att(output)
        # [1, hidden*2]
        output = self.self_att_2(output.unsqueeze(0))

        # [candidate_len, hidden*2]
        c_out = self.self_att_c(c_out)
        
        # score layer
        score = torch.mm(c_out, torch.tanh(output.transpose(0, 1)))
        return score

In [10]:
class Config:
    
    def __init__(self):
        self.hidden = 100
        self.word_dim = 300
        self.bert_dim = 768
        self.use_cuda = True
        self.bert_path = './bert-base-uncased/'
        self.use_all = True
        self.lr = 1e-4
        self.epochs = 30
        self.log_dir = './logs'
        self.model_name = 'simpleQANet'
        self.batch_size = 8

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
def train(epoch, data_iter, model, criterion, optimizer, cuda, batch_size=1):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()
    #model.embedding_layer.eval()
    for idx, batch in enumerate(data_iter):
        score = model(batch)
        label = batch.label
        if cuda:
            label = label.cuda()
        score = score.transpose(0,1)      
        
        loss = criterion(score, label)

        loss = loss / batch_size
        loss.backward()
        if (idx+1)%batch_size == 0 :
            optimizer.step()
            optimizer.zero_grad()        

        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()   
        acces.update(acc)
        if (idx+1) % (batch_size*100) == 0:
            print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

def val(epoch, data_iter, model, criterion, cuda):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()
    for idx, batch in enumerate(data_iter):
        with torch.no_grad():
            score = model(batch)
            
        label = batch.label
        if cuda:
            label = label.cuda()
        score = score.transpose(0,1)      
        
        loss = criterion(score, label)
        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()   
        acces.update(acc)
        if idx % 100 == 0:
            print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

In [13]:
config = Config()
model = SimpleQANet(config, word_field)
if config.use_cuda:
    model = model.cuda()

In [14]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                             lr=config.lr)

criterion = nn.CrossEntropyLoss()

In [15]:
save_path = config.model_name + '_epoch'+str(config.epochs) + '_lr'+ str(config.lr)+ '_useall'+ \
                str(config.use_all) + '_batchsize' + str(config.batch_size)

save_path = os.path.join(config.log_dir, save_path)
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
print(save_path)

writer = SummaryWriter(save_path)

./logs/simpleQANet_epoch30_lr0.0001_useallTrue_batchsize8


In [ ]:
best_acc = 0.0
for epoch in range(config.epochs):
    train_loss, train_acc = train(epoch, train_iter, model, criterion, optimizer, 
                                  config.use_cuda, config.batch_size)
    val_loss, val_acc = val(epoch, val_iter, model, criterion, config.use_cuda)
    
    writer.add_scalar('train_loss', train_loss, epoch+1)
    writer.add_scalar('val_loss', val_loss, epoch+1)
    writer.add_scalar('train_acc', train_acc, epoch+1)
    writer.add_scalar('val_acc', val_acc, epoch+1)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_path, 'best.pth'))

/home/user2/Multi_QA/model.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)


epoch:0, idx:799/43738, loss:0.28659709547646345, acc:0.265
epoch:0, idx:1599/43738, loss:0.2743978534586495, acc:0.296875
epoch:0, idx:2399/43738, loss:0.27228552452870647, acc:0.3070833333333333
epoch:0, idx:3199/43738, loss:0.268721807145339, acc:0.3184375
epoch:0, idx:3999/43738, loss:0.26731599562172775, acc:0.32175
epoch:0, idx:4799/43738, loss:0.26574684386413233, acc:0.3258333333333333
epoch:0, idx:5599/43738, loss:0.2635271064802406, acc:0.3291071428571429
epoch:0, idx:6399/43738, loss:0.261661778416601, acc:0.3309375
epoch:0, idx:7199/43738, loss:0.25964686856229996, acc:0.3347222222222222
epoch:0, idx:7999/43738, loss:0.25868207126075865, acc:0.337125
epoch:0, idx:8799/43738, loss:0.25706368385930545, acc:0.3427272727272727
epoch:0, idx:9599/43738, loss:0.2563256980104294, acc:0.34510416666666666
epoch:0, idx:10399/43738, loss:0.2555852147268328, acc:0.34615384615384615
epoch:0, idx:11199/43738, loss:0.2546444391916988, acc:0.34794642857142855
epoch:0, idx:11999/43738, loss:

epoch:1, idx:7999/43738, loss:0.2055365326437168, acc:0.456125
epoch:1, idx:8799/43738, loss:0.2057263868378306, acc:0.4553409090909091
epoch:1, idx:9599/43738, loss:0.20574462676692443, acc:0.45489583333333333
epoch:1, idx:10399/43738, loss:0.20619887379571222, acc:0.4530769230769231
epoch:1, idx:11199/43738, loss:0.20683590275767658, acc:0.45026785714285716
epoch:1, idx:11999/43738, loss:0.2065541827729903, acc:0.4513333333333333
epoch:1, idx:12799/43738, loss:0.20572538802487542, acc:0.455078125
epoch:1, idx:13599/43738, loss:0.2055854808707612, acc:0.45544117647058824
epoch:1, idx:14399/43738, loss:0.20571503492777185, acc:0.4542361111111111
epoch:1, idx:15199/43738, loss:0.2056441273185498, acc:0.45480263157894735
epoch:1, idx:15999/43738, loss:0.205246537637664, acc:0.4558125
epoch:1, idx:16799/43738, loss:0.20491843341478325, acc:0.45613095238095236
epoch:1, idx:17599/43738, loss:0.2045202373347605, acc:0.4575568181818182
epoch:1, idx:18399/43738, loss:0.20404567944619845, acc:0

epoch:2, idx:15199/43738, loss:0.18893969198186442, acc:0.4869078947368421
epoch:2, idx:15999/43738, loss:0.18883445119240788, acc:0.4870625
epoch:2, idx:16799/43738, loss:0.18826402264863384, acc:0.4892261904761905
epoch:2, idx:17599/43738, loss:0.18816463076028, acc:0.4900568181818182
epoch:2, idx:18399/43738, loss:0.18777470174528982, acc:0.49130434782608695
epoch:2, idx:19199/43738, loss:0.18751743714334831, acc:0.4925
epoch:2, idx:19999/43738, loss:0.18747297888682224, acc:0.4933
epoch:2, idx:20799/43738, loss:0.18708908152139675, acc:0.49514423076923075
epoch:2, idx:21599/43738, loss:0.18715216019432301, acc:0.49527777777777776
epoch:2, idx:22399/43738, loss:0.1872454044004969, acc:0.4959375
epoch:2, idx:23199/43738, loss:0.18725957362992882, acc:0.49530172413793105
epoch:2, idx:23999/43738, loss:0.18718895005361022, acc:0.49525
epoch:2, idx:24799/43738, loss:0.1869849894380182, acc:0.4958467741935484
epoch:2, idx:25599/43738, loss:0.18661253569352992, acc:0.497109375
epoch:2, id

epoch:3, idx:22399/43738, loss:0.17387903640537844, acc:0.5340178571428571
epoch:3, idx:23199/43738, loss:0.17414201459101228, acc:0.5335344827586207
epoch:3, idx:23999/43738, loss:0.17390148590622023, acc:0.5343333333333333
epoch:3, idx:24799/43738, loss:0.17353586397134238, acc:0.535
epoch:3, idx:25599/43738, loss:0.1733046379653797, acc:0.535546875
epoch:3, idx:26399/43738, loss:0.17329811796362982, acc:0.5358333333333334
epoch:3, idx:27199/43738, loss:0.1733474590874539, acc:0.5358455882352942
epoch:3, idx:27999/43738, loss:0.17305401336648252, acc:0.5367142857142857
epoch:3, idx:28799/43738, loss:0.1731525816052494, acc:0.536875
epoch:3, idx:29599/43738, loss:0.17339660238733, acc:0.5362837837837838
epoch:3, idx:30399/43738, loss:0.1732394272435225, acc:0.53625
epoch:3, idx:31199/43738, loss:0.17277638978766985, acc:0.5371794871794872
epoch:3, idx:31999/43738, loss:0.17291489277527217, acc:0.53703125
epoch:3, idx:32799/43738, loss:0.1727415617793073, acc:0.5373780487804878
epoch:3

...

epoch:22, idx:1900/5129, loss:1.274652397743123, acc:0.6354550236717517
epoch:22, idx:2000/5129, loss:1.2689469087047913, acc:0.6356821589205397
epoch:22, idx:2100/5129, loss:1.27093912123116, acc:0.6354117087101381
epoch:22, idx:2200/5129, loss:1.267229222038804, acc:0.6369831894593366
epoch:22, idx:2300/5129, loss:1.2678497424498945, acc:0.6384180790960452
epoch:22, idx:2400/5129, loss:1.270567241821846, acc:0.6347355268638067
epoch:22, idx:2500/5129, loss:1.277948364445635, acc:0.6337465013994402
epoch:22, idx:2600/5129, loss:1.2897055246385645, acc:0.6309111880046137
epoch:22, idx:2700/5129, loss:1.2878311027430331, acc:0.6305072195483155
epoch:22, idx:2800/5129, loss:1.284063976692945, acc:0.6301320956801142
epoch:22, idx:2900/5129, loss:1.2860917394781104, acc:0.6294381247845571
epoch:22, idx:3000/5129, loss:1.2981657828565878, acc:0.6284571809396867
epoch:22, idx:3100/5129, loss:1.2984216311277976, acc:0.6281844566268946
epoch:22, idx:3200/5129, loss:1.3025790029919657, acc:0.62

epoch:23, idx:2700/5129, loss:1.2931565225554678, acc:0.6227323213624584
epoch:23, idx:2800/5129, loss:1.294333087620204, acc:0.6219207425919314
epoch:23, idx:2900/5129, loss:1.2968550335997262, acc:0.6204756980351603
epoch:23, idx:3000/5129, loss:1.3083430012090331, acc:0.6201266244585139
epoch:23, idx:3100/5129, loss:1.306771341181501, acc:0.6214124475975492
epoch:23, idx:3200/5129, loss:1.312887499780999, acc:0.6188691034051859
epoch:23, idx:3300/5129, loss:1.3122184261277892, acc:0.6192063011208725
epoch:23, idx:3400/5129, loss:1.2981731053775845, acc:0.6236401058512202
epoch:23, idx:3500/5129, loss:1.2961743509716661, acc:0.6243930305626963
epoch:23, idx:3600/5129, loss:1.2871932142006295, acc:0.6262149402943626
epoch:23, idx:3700/5129, loss:1.2908450818221269, acc:0.6263172115644421
epoch:23, idx:3800/5129, loss:1.282946305972311, acc:0.6272033675348593
epoch:23, idx:3900/5129, loss:1.2875400352855426, acc:0.6257369905152524
epoch:23, idx:4000/5129, loss:1.2882203919288369, acc:0

epoch:24, idx:3400/5129, loss:1.3032592700986574, acc:0.6218759188473978
epoch:24, idx:3500/5129, loss:1.2982227306677865, acc:0.6246786632390745
epoch:24, idx:3600/5129, loss:1.28899323077905, acc:0.627048042210497
epoch:24, idx:3700/5129, loss:1.291652356924414, acc:0.6265874088084301
epoch:24, idx:3800/5129, loss:1.2827170698917467, acc:0.6277295448566167
epoch:24, idx:3900/5129, loss:1.287300544032926, acc:0.6267623686234299
epoch:24, idx:4000/5129, loss:1.2866786926262261, acc:0.6263434141464633
epoch:24, idx:4100/5129, loss:1.2900618286183392, acc:0.6264325774201415
epoch:24, idx:4200/5129, loss:1.2831668809705392, acc:0.6269935729588193
epoch:24, idx:4300/5129, loss:1.2783541084929584, acc:0.627993489886073
epoch:24, idx:4400/5129, loss:1.278407408070548, acc:0.6284935241990457
epoch:24, idx:4500/5129, loss:1.2717913345042082, acc:0.629193512552766
epoch:24, idx:4600/5129, loss:1.2737205542123622, acc:0.6276896326885459
epoch:24, idx:4700/5129, loss:1.2768659782178908, acc:0.626

epoch:25, idx:4200/5129, loss:1.2827679892557202, acc:0.6279457272078076
epoch:25, idx:4300/5129, loss:1.2794618642027364, acc:0.6286910020925366
epoch:25, idx:4400/5129, loss:1.2801740032854987, acc:0.6291751874573961
epoch:25, idx:4500/5129, loss:1.2721651442244672, acc:0.6311930682070651
epoch:25, idx:4600/5129, loss:1.2735498618788756, acc:0.6300804173005868
epoch:25, idx:4700/5129, loss:1.2773935184567387, acc:0.62965326526271
epoch:25, idx:4800/5129, loss:1.2725886995765447, acc:0.6296604873984587
epoch:25, idx:4900/5129, loss:1.2757140334404455, acc:0.629463374821465
epoch:25, idx:5000/5129, loss:1.2787444631196074, acc:0.6286742651469706
epoch:25, idx:5100/5129, loss:1.275517024821369, acc:0.6288962948441482
epoch:26, idx:799/43738, loss:0.08557836972875521, acc:0.76125
epoch:26, idx:1599/43738, loss:0.08314809171133675, acc:0.774375
epoch:26, idx:2399/43738, loss:0.08359731910633855, acc:0.7720833333333333
epoch:26, idx:3199/43738, loss:0.08584918272827054, acc:0.766875
epoch:

epoch:26, idx:5000/5129, loss:1.2927229169192278, acc:0.6280743851229754
epoch:26, idx:5100/5129, loss:1.2884516166828002, acc:0.6277200548911978
epoch:27, idx:799/43738, loss:0.07233596364210826, acc:0.805
epoch:27, idx:1599/43738, loss:0.07774412773171208, acc:0.79125
epoch:27, idx:2399/43738, loss:0.07706241062958724, acc:0.7916666666666666
epoch:27, idx:3199/43738, loss:0.07723759949018132, acc:0.7925
epoch:27, idx:3999/43738, loss:0.07848630557616706, acc:0.78975
epoch:27, idx:4799/43738, loss:0.08096730528738894, acc:0.7795833333333333
epoch:27, idx:5599/43738, loss:0.08157123045289025, acc:0.7769642857142857
epoch:27, idx:6399/43738, loss:0.08128193423523043, acc:0.77734375
epoch:27, idx:7199/43738, loss:0.08181795872243432, acc:0.7747222222222222
epoch:27, idx:7999/43738, loss:0.08221371719695161, acc:0.77425
epoch:27, idx:8799/43738, loss:0.08237324438010216, acc:0.772159090909091
epoch:27, idx:9599/43738, loss:0.0823893969206741, acc:0.7729166666666667
epoch:27, idx:10399/437

In [ ]:
layer = model.linear_1[0]

In [ ]:
model.train()
cuda = True
#model.embedding_layer.eval()
for idx, batch in enumerate(train_iter):
    score = model(batch)
    label = batch.label
    if cuda:
        label = label.cuda()
    score = score.transpose(0,1)      

    loss = criterion(score, label)

    optimizer.zero_grad()
    loss.backward()
    # optimizer.step()
    break